<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Only-change-this,-the-input-file" data-toc-modified-id="Only-change-this,-the-input-file-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Only change this, the input file</a></span></li></ul></div>

# Imports

In [10]:
# Plotting tools
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import re
import numpy as np
import pandas as pd

#NLTK
import nltk
# nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel

# spacy for lemmatization
import spacy
if 'nlp' not in locals():
#     nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
    nlp = spacy.load('en', disable=['parser', 'ner'])

import gc
import os
from glob import glob
from IPython.display import Audio, display

# Text Analysis (uncomment if running for first time)
# ! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# ! unzip mallet-2.0.8.zip
MALLET_PATH = 'mallet-2.0.8/bin/mallet'

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from pathlib import *
#current working directory
current_dir = Path.cwd()
#go up 1 level to the 1st parent directory
Par1_dir = current_dir.parents[0]

# Alert Function
def allDone():
    '''this function outputs a short audio when called. 
    Typically this is used to signal a task completion'''
    
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

# Only change this, the input file

In [11]:
# What folder is it in, 'ai' or 'dl'
ai_or_dl = 'ai'
df_filename = 'globe_#ML_en_130101_190314_3.csv'

In [12]:
df_data_dir = Par1_dir /'AI Project'/'Python Env' / 'DataFrames'/ 'AI_datasets' / 'globe_ai_13_18'/ df_filename

#import only the required columns from the DF
df = pd.read_csv(df_data_dir, usecols=['tweet', 'date/time', 'search'])

#find dataframe's search word, and remove quotations f present
search_word = df.head(1).search.item().replace('"', '').replace("'","")
#drop unused columns
df = df.loc[:, ['tweet', 'date/time']]

df_list = []
# #create a random Sample of 100k for each year & drop NAs
for x in range(2013,2019):
    globals()['df_%s' % x] = df.loc[(df['date/time']>=str(x)+'-01-01 00:00:00+00:00') & (df['date/time']<str(x+1)+'-01-01 00:00:00+00:00'),: ].copy()
    n = min(100000, len(globals()['df_%s' % x]))
    globals()['df_%s' % x] = globals()['df_%s' % x].sample(n=n, random_state=11).dropna(subset=['tweet'])
    df_list.append(globals()['df_%s' % x])
    del globals()['df_%s' % x]

#delete the big DF
del df
gc.collect()


/anaconda3/envs/ai_project/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: `item` has been deprecated and will be removed in a future version
  import sys


104

In [13]:
df = pd.concat(df_list)
search_word

'#ML'

In [14]:
# NLTK Stop words
from nltk.corpus import stopwords

#NLTK english stopwords
stop_words = stopwords.words('english')
#extend the list with a peronal list of stopwords
stop_words.extend(['from', 'need','thank','thing','something', 'see', 'say', 'well','people', 'change', 'com',\
                   'go', 'put', 'give','twitter','pic', \
                   'subject', 're', 'edu', 'could', 'be', 'make', 'not', 'make','find','let','may','see', 'would',\
                   'come', 'sure', 'ever', 'tell', 'use', 'not', 'doing', 'be', 'get','want'])
#extend the search word
stop_words.extend(['artificial intelligence', '#ai', '#ml', '#nlp', 'analytics', 'data mining',
                  'deep mining', 'machine learning', 'natural language processing', 'neural network'
                  'pattern recognition'])

### Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
###Remove Stopwords, Make Bigrams and Lemmatize
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(bigram_mod, texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


def lda_preprocessing(df):
    ### Remove emails, newline characters, and links
    # Convert to list
    data = df.tweet.values.tolist()

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', str(tweet)) for tweet in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', tweet) for tweet in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", tweet) for tweet in data]

    #Remove links
    data = [re.sub(r"http\S+", "", tweet) for tweet in data]

    #make lower case
    data = [tweet.lower() for tweet in data]
    
    # Tokenize words and Clean-up text
    data_words = list(sent_to_words(data))

    ###Creating Bigram and Trigram Models
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(bigram_mod, data_words_nostops)

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN','ADJ','ADV', 'VERB'])
    data_lemmatized = remove_stopwords(data_lemmatized)

    ###Create the Dictionary and Corpus needed for Topic Modeling¶
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

        
    return data_lemmatized, corpus, id2word

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.wrappers import LdaMallet
from gensim.models.wrappers.ldamallet import malletmodel2ldamodel
from multiprocessing import cpu_count

#     df = df_list[1]

data_lemmatized, CORPUS, id2word = lda_preprocessing(df)

#get current working directory
cwd = os.getcwd()

#set the number of workers to the number of cores available
workers = cpu_count() 

# Set number of topics
NUM_TOPICS = 20

ldamallet = LdaMallet(MALLET_PATH, corpus=CORPUS, num_topics=NUM_TOPICS, id2word=id2word,
                                  random_seed =11,workers=workers)

# Transform model weights from MALLET to GENSIM
lda_model = malletmodel2ldamodel(ldamallet)

# Save Topic meanings
topic_content = pd.DataFrame(ldamallet.print_topics(num_topics=NUM_TOPICS, num_words=100))
topic_content = topic_content.rename(columns={0:'topic_n', 1:'content'})

filename = 'topic_content_' + search_word + '.csv'
foldername = 'pca_data'
save_dir = Par1_dir /'AI Project'/'Avery_output' / foldername / ai_or_dl / filename
topic_content.to_csv(save_dir, index=False)

############################
# Get doc-topic probability#
############################

# get transformed corpus as per the LDA model
TRANSF_CORPUS = lda_model.get_document_topics(CORPUS)

# rearrange data on document-topic pairs probabilities
DOC_TOPIC_M = []

for id, doc in enumerate(TRANSF_CORPUS):
    for topic in np.arange(0, 20, 1):
        topic_n = doc[topic][0]
        topic_prob = doc[topic][1] 
        DOC_TOPIC_M.append([id, topic, topic_prob])

DF = pd.DataFrame(DOC_TOPIC_M)

# rename columns
OLD_NAMES = [0, 1, 2]
NEW_NAMES = ['doc_id', 'topic_n', 'prob']
COLS = dict(zip(OLD_NAMES, NEW_NAMES))
DF.rename(columns=COLS, inplace=True)

DF1 = DF.copy() # Checkpoint
DF1 = DF1.pivot_table(index='doc_id', columns='topic_n', values='prob', aggfunc=np.mean)
DF1 = DF1.reset_index()

# Keep only dominant topic for vis
GR = DF.groupby('doc_id')
DF.loc[:, 'max'] = GR['prob'].transform(np.max)
DF.loc[:, 'first_topic'] = 0
DF.loc[DF['prob'] == DF['max'], 'first_topic'] = 1
FIRST_TOPIC = DF.loc[DF['first_topic'] == 1]

# Record Dates
dates = pd.DataFrame()
dates['date'] = df['date/time']
dates['year'] = dates['date'].str.split('-', expand=True)[0]
dates = dates.reset_index(drop=True) 
dates = dates.reset_index()
dates = dates.rename(columns={'index':'doc_id'})

# Merge to keep the First order
DF2 = pd.merge(DF1, FIRST_TOPIC, how='left', on='doc_id')
DF2 = pd.merge(DF2, dates, how='left', on='doc_id') 

# write data to file
filename = search_word + '.csv'
foldername = 'pca_data'
save_dir = Par1_dir /'AI Project'/'Avery_output' / foldername / ai_or_dl / filename
DF2.to_csv(save_dir, index=False)

def pca_data(df):

    df1 = df.loc[:, '0':'19']
    X = df1.to_numpy()

    df2 = df.drop(columns=df1.columns)
    df2 = df2[['topic_n', 'prob', 'date', 'year']]
    scaler = StandardScaler() # instantiate
    scaler.fit(X) # compute the mean and standard which will be used in the next command
    X_scaled = scaler.transform(X)
    # fit and transform can be applied together and I leave that for simple exercise
    # we can check the minimum and maximum of the scaled features which we expect to be 0 and 1
    # print ("after scaling minimum", X_scaled.min(axis=0))
    pca = PCA(n_components=2)
    pca.fit(X_scaled)
    X_pca = pca.transform(X_scaled)
    vdf = pd.DataFrame()
    vdf['x'] = X_pca[:,0]
    vdf['y'] = X_pca[:,1]
    # vdf = vdf.reset_index()
    # vdf = vdf.rename(columns={'index':'doc_id'})
    vdf = pd.merge(vdf, df2, how='left', left_index=True, right_index=True)

    return vdf

DF3 = pd.read_csv(save_dir)
vdf = pca_data(DF3)
filename = 'vis_ready_' + search_word + '.csv'
foldername = 'pca_data'
save_dir = Par1_dir /'AI Project'/'Avery_output' / foldername / ai_or_dl / filename
vdf.to_csv(save_dir, index=False)


allDone()